## 0. 라이브러리 다운로드

In [35]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !pip install git+https://github.com/haven-jeon/PyKoSpacing.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-7_ga9ot2
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-7_ga9ot2
     |████████████████████████████████| 511.8 MB 24 kB/s 
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268637 sha256=9a3aaa30825b75a3f50f2ac47505b213dff1c3cf244da925900a08eedf8f4965
  Stored in directory: /tmp/pip-ephem-wheel-cache-e2dqpdpl/wheels/79/a0/33/16f2cd03d21f76a663f5d69a0b96f0351335385349136fbd03
Successfully built pykospacing
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2


In [36]:
import re
import time
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from konlpy.tag import Kkma
# from pykospacing import Spacing
from collections import Counter
import numpy as np
import pandas as pd

## 1. 데이터 불러오기
* 다섯개 구 합쳐진 total 데이터 사용

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AIVLE_BigProject/data/total_total.csv")

In [ ]:
df.shape

(12724, 53)

In [ ]:
df.head()

,search_key,name_x,category,rating,address,tel,year_10,year_20,year_30,year_40,...,review_blog1,review_blog2,review_blog3,review_blog4,review_blog5,review_blog6,review_blog7,review_blog8,review_blog9,review_blog10
0,한식,떡도리탕 강남본점,닭볶음탕,4.47,서울 강남구 테헤란로1길 28-9 1층,0507-1352-2940,29.55,75.06,49.99,28.73,...,강남 맛집 풍자가 선택한 또간집 떡도리탕 내돈내산 후기|회사 점심시간에 유튜버이자 ...,11.28~12.4 주간 일기|벌써 주간 일기가 마지막 주간이구나. 이렇게 오래 쓰...,강남역 맛집 떡도리탕 주말 웨이팅 성공|강남맛집 떡도리탕입니다. 한 번의 실패를 맛...,[서울] 떡도리탕 강남본점 - 강남역 신논현역 닭볶음탕 + 떡볶이|언니야가 한번 가...,강남 맛집 또간집 떡도리탕 웨이팅|안녕하세요 재교입니다. 강남 맛집 또간집 떡도리탕...,[내돈내산] 강남 떡도리탕 풍자 또간집 맛집 방문 웨이팅 후기|오랜만에 강남에서 계...,떡도리탕 강남본점 또간집 웨이팅후기|안녕하세요 코코입니다 평소 유튜브로 또간집을 즐...,강남역 맛집 떡도리탕 평일 웨이팅 꿀팁 / 풍자 또간집|오늘은 요즘 강남역의 핫한 ...,강남 떡도리탕 평일 웨이팅 예약방법 /풍자 또간집 맛집|최근에 관심있게 보는 유튜브...,강남 떡도리탕 오픈런 성공후기 :)|어제는 친구들을 만났지. 출산전에 꼭 한번 만나...
1,한식,도명골 청국장,"찌개,전골",4.61,서울 강남구 봉은사로114길 38 101호 도명골 청국장,0507-1457-7015,11.18,51.40,69.13,78.08,...,삼성역 맛집 성시경 먹을텐데 도명골청국장 웨이팅|안녕하세요 재교입니다. 삼성역 맛집...,평일 오프 #23|오늘 개봉하는 기대작 '아바타 : 물의 길' '아이맥스'나/'돌비...,삼성역 맛집 도명골청국장 - 성시경 먹을텐데 삼성동 청국장|#도명골청국장 #삼성역맛...,성시경 먹을텐데 가성비 좋은 강남 삼성동맛집 청국장 맛집 도명골... |요즘 유튜브...,"[삼성역] 도명골청국장 ""청국장""|오늘 저녁 한끼는 삼성역에 있는 도명골청국장에서 ...",도담골청국장 성시경먹을텐데 도장깨기!!추운날 딱인곳~|안녕하세요 연남동 꽃녀 입니다...,[삼성] 오늘은 '도명골 청국장'에서 먹었다|삼성역에 근처에 외근 나가는 날이어서 ...,"[삼성] 세상 푸짐한 도명골 청국장! | 성시경 먹을텐데 맛집, 청국장... |#도...",[삼성동 존맛/맛집] - 도명골 청국장 (성시경의 먹을텐데)|이 포스팅은 제 지갑과...,"코엑스, 봉은사, 서울의료원 맛집 도명골 청국장|코엑스 전시회 보러갔다가 배고파서 ..."
2,한식,자매수산,생선회,4.30,서울 강남구 봉은사로18길 90,0507-1309-5534,8.82,97.53,82.21,59.64,...,[강남/신논현] 자매수산 - 겨울을 맞이한 방어회 (그리고 물건너간... |안뇽하세...,강남역 방어맛집 존맛이네 :: 자매수산 1호점|안녕하세요 리야입니다. 방어의 계절이...,"강남역 자매수산 방문, 자매 제철 스페셜(大) & 매운탕|강남역 자매 수산 1호점 ...",[강남] 겨울 방어회/서울 5대 대방어맛집_자매수산|유유맛집 pick 자매수산 매일...,"강남역 방어 / 자매수산 / 웨이팅, 예약 등 정보|오늘의 먹부림은 바로 강남역 방...",[강남 핫플] 자매 수산 대방어 | 포토그레이 | 강남 포장마차 | 떡볶이|불토 뜨...,강남역 방어회 맛집 자매수산 평일 저녁 웨이팅|안녕하세요 띠니입니다. 강남역 저녁...,강남역 횟집 서울 5대 대방어 맛집이라는 자매수산 1호점 솔직후기|강남역 횟집 서울...,강남역 맛집 베스트10_가을 회먹기 좋은 자매수산|강남역 맛집 베스트10_가을 회먹...,"강남 대방어 맛집 ""자매수산"" 리뷰(웨이팅 및 메뉴추천 등)|대 방어 맛집으로 유명..."
3,한식,삼백호집 신논현점,"육류,고기요리",0.00,서울 강남구 봉은사로1길 27 1층,0507-1433-0292,4.26,34.80,31.77,31.17,...,신논현맛집) 가성비로 우릴 사로잡는 고기맛집 삼백호집|고기는 언제 먹어도 맛있다. ...,신논현고기집 강남 가성비 추천 신논현역맛집 삼백호집|환절기엔 삼겹살로 상쾌하게 보내...,신논현고기집 가성비 맛집 삼백호집 고기잔치 메뉴 구성 너무 좋네요|신논현고기집 가성...,신논현맛집 가성비 갓 백호 돼지고기잔치 먹으러 삼백호집|안녕하세요 변스텝입니다. 신...,"[강남] 삼백호집 | 신논현맛집 가성비 고깃집 추천(메뉴, 세트구성)|신논현, 논현...",강남 신논현 점심 추천 삼백호집|신논현고깃집#신논현맛집#신논현점심 점심 먹으러 기웃...,"신논현고기집 삼백호집, 고기로 배 채울 수 있는 가성비 맛집|신논현역 먹자골목에 위...",신논현맛집 미친 가성비의 숙성고기집 삼백호집 후기|안녕하세요 한창 벌크업 중인 여자...,강남 봉은사로 논현 삼백호집 고기 가성비 최고 맛집 고기로 배 채우는 집|#논현삼백...,신논현역 점심 메뉴 삼백호집 김치찌개(투어큐레이터)|신논현역 점심 메뉴 삼백호집 김...
4,한식,대우부대찌개,"찌개,전골",4.42,서울 강남구 테헤란로25길 34,02-552-1663,17.77,63.06,78.44,80.43,...,역삼동 맛집 대우부대찌개 본점 (feat. 성시경 먹을텐데)|Copyright ⓒ ...,역삼역 맛집 대우부대찌개 / 막돼먹은 비주얼로 강남을 제패한 부찌|자극적이지 않은 ...,"역삼맛집 성시경의 먹을텐데 소개된 대우부대찌개, 등심과 안심구이|강남 역삼맛집 대우...",고기맛이 찐~했던 역삼 <대우 부대찌개> 내돈내산|대우 부대찌개는 회사 맛집 WIK...,성시경 '먹을텐데' 도장깨기! 역삼동 대우 부대찌개 먹고왔어요!... |역삼동 대우...,[역삼역점심맛집/대우부대찌개] - 역삼 직장인들의 소울푸드|역삼역은 직장인들이 많다...,강남 역삼동 맛집 '대우부대찌개' 토요일은밥이좋아 한우등심부대찌개... |서울 역삼...,[역삼동 맛집] 대우부대찌개 - 부대찌개 생각날 때마다 가는 곳|오랜만에 방문한 대...,역삼동 대우부대찌개 - 먹을텐데 강남 부대찌개|#대우부대찌개 2022년 11월 17...,11111111111월 추워요|11월 뭐가 그렇게 귀찮았을까 몰아 쓰는 11월 일...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12724 entries, 0 to 12723
Data columns (total 53 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   search_key            12724 non-null  object 
 1   name_x                12724 non-null  object 
 2   category              12724 non-null  object 
 3   rating                12724 non-null  float64
 4   address               12724 non-null  object 
 5   tel                   11152 non-null  object 
 6   year_10               12724 non-null  float64
 7   year_20               12724 non-null  float64
 8   year_30               12724 non-null  float64
 9   year_40               12724 non-null  float64
 10  year_50               12724 non-null  float64
 11  year_60               12724 non-null  float64
 12  male                  12724 non-null  int64  
 13  female                12724 non-null  int64  
 14  detail_url            12724 non-null  object 
 15  gu                 

## 2-1. 텍스트 데이터 전처리
* 리뷰 데이터 10개 하나로 합치기(가게별)

In [ ]:
temp = df[['review1', 'review2', 'review3', 'review4', 'review5', 'review6', 'review7', 'review8', 'review9', 'review10']]

In [ ]:
temp.loc[temp['review1'].isna()]    ## 리뷰가 아예 없는 애들이 있음. 일단 이대로 공란으로 두고 없으면 안 보여주는 식..?

,review1,review2,review3,review4,review5,review6,review7,review8,review9,review10
417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
12717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
### review 데이터 10개 한번에 합치기(가게별로) : 새 칼럼 생성

r = []
for i in range(len(df)):
    pre = []
    for j in range(10):
        col = "review" + str(j+1)
        pre.append(str(df.iloc[i][col]))
        full_review = ' '.join(pre)
    r.append(full_review)

df["reviews"] = r

In [ ]:
df['review1'][0]

'엄~~~청 기대하고 갔는데...이렇게 줄까지 서서 먹을만한 곳은 아니라는 생각이 들었습니다!음...엄청 달고 짜고..그래도 떡은 진짜 맛있더라고요!  볶음밥도 괜찮았습니다! 좀 칼칼하고 깔끔한 닭볶음탕 찾으시는 분들은 다른 곳 추천드려요~   '

In [ ]:
df['reviews'][0]

'엄~~~청 기대하고 갔는데...이렇게 줄까지 서서 먹을만한 곳은 아니라는 생각이 들었습니다!음...엄청 달고 짜고..그래도 떡은 진짜 맛있더라고요!  볶음밥도 괜찮았습니다! 좀 칼칼하고 깔끔한 닭볶음탕 찾으시는 분들은 다른 곳 추천드려요~    지난번엔 줄이 길어 실패했는데, 이번 방문엔 성공했네요~양도 많고 맛있어요~양념이 너무 맛있어서 밥 비벼 먹으면 최고~!추운날 얼큰한 국물 생각날때 좋아요~기본 양이 많아 라면사리랑 볶음밥을 못 먹은게 너무 아쉽네요~다 먹고 너무 맛있어서 집에 포장도 해왔어요~!! 벌써 두번째 방문!\n또간집 보고 방문했어요\n소스가 정말 미쳤어요! 너무 맛있어요ㅠ\n다른 테이블에서도 다들 감탄소리뿐였어요\n닭에 소스가 베이는 동안 먹는 떡도 최고예오\n밀떡을 좋아하는데 간이 쏙쏙 베어들어 아주 환상이랍니다!  감자도 정말 맛나는 감자예요 소스에 으깨먹으면 진짜 환장네이션\n닭살들도 정말 야들하고 냄새 안나요!\n사장님 직원들 너무나도 친절하시구 좋습니다\n볶음밥은 꼭 드시고 가셔야 해요\n여기 참기름을 좋은 걸 쓰시는지.. 고소한 맛과 소스와 밥이 아주 환상적.. \n볶음밥이랑 깍두기를 꼭 드셔야 해요 이 볶음밥을 위해 만들어진 깍두기인지 정말 찰떡궁합이랍니다.. 콩나물무침은 말할 것도 없거요.. 정말 강남 최고의 맛집.. 추천입니다\n단 테이블링은 꼭 하고 가셔야 해요.. 수요일 저녁 5시 10분에 테이블링 원격줄서기 걸고 8시에 입장해서 먹었어요! 텁텁함 1도 없이 깔끔하고 맛있는 떡도리탕이었어요! 약간 떡볶이 느낌이 더강한!? 감자전은 감자튀김전같아서 조금 실망했네요ㅠㅠ 밑반찬 콩나물이 증맬 맛있어요! 또간집 나오기전에도 왔었는데 맛도리인걸 알았지만 또간집에까지 나와서 이렇게 많아질줄몰랐어요.\n주말 4시반정도방문했는데 앞에 24팀 대기있었고 5시까지 브레이크타임이었지만 4시 40분좀넘어서부터 들어오게해주시더라구요.\n매장이 꽤나 넓은편이라 저희도 다행히 5시다되서(한사이클 돌기전) 입장할 수 있었어요. \n빠른회전을 위해서인

## 2-2. 텍스트 데이터 전처리
* 한글만 남기기
* 띄어쓰기 교정

In [ ]:
import re

## 한글만 남기기
def text_cleaning(text):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result

In [ ]:
df['ko_text'] = df['reviews'].apply(lambda x: text_cleaning(x))
# del temp['reviews']
df.head()

In [ ]:
df['reviews'][0]

'엄~~~청 기대하고 갔는데...이렇게 줄까지 서서 먹을만한 곳은 아니라는 생각이 들었습니다!음...엄청 달고 짜고..그래도 떡은 진짜 맛있더라고요!  볶음밥도 괜찮았습니다! 좀 칼칼하고 깔끔한 닭볶음탕 찾으시는 분들은 다른 곳 추천드려요~    지난번엔 줄이 길어 실패했는데, 이번 방문엔 성공했네요~양도 많고 맛있어요~양념이 너무 맛있어서 밥 비벼 먹으면 최고~!추운날 얼큰한 국물 생각날때 좋아요~기본 양이 많아 라면사리랑 볶음밥을 못 먹은게 너무 아쉽네요~다 먹고 너무 맛있어서 집에 포장도 해왔어요~!! 벌써 두번째 방문!\n또간집 보고 방문했어요\n소스가 정말 미쳤어요! 너무 맛있어요ㅠ\n다른 테이블에서도 다들 감탄소리뿐였어요\n닭에 소스가 베이는 동안 먹는 떡도 최고예오\n밀떡을 좋아하는데 간이 쏙쏙 베어들어 아주 환상이랍니다!  감자도 정말 맛나는 감자예요 소스에 으깨먹으면 진짜 환장네이션\n닭살들도 정말 야들하고 냄새 안나요!\n사장님 직원들 너무나도 친절하시구 좋습니다\n볶음밥은 꼭 드시고 가셔야 해요\n여기 참기름을 좋은 걸 쓰시는지.. 고소한 맛과 소스와 밥이 아주 환상적.. \n볶음밥이랑 깍두기를 꼭 드셔야 해요 이 볶음밥을 위해 만들어진 깍두기인지 정말 찰떡궁합이랍니다.. 콩나물무침은 말할 것도 없거요.. 정말 강남 최고의 맛집.. 추천입니다\n단 테이블링은 꼭 하고 가셔야 해요.. 수요일 저녁 5시 10분에 테이블링 원격줄서기 걸고 8시에 입장해서 먹었어요! 텁텁함 1도 없이 깔끔하고 맛있는 떡도리탕이었어요! 약간 떡볶이 느낌이 더강한!? 감자전은 감자튀김전같아서 조금 실망했네요ㅠㅠ 밑반찬 콩나물이 증맬 맛있어요! 또간집 나오기전에도 왔었는데 맛도리인걸 알았지만 또간집에까지 나와서 이렇게 많아질줄몰랐어요.\n주말 4시반정도방문했는데 앞에 24팀 대기있었고 5시까지 브레이크타임이었지만 4시 40분좀넘어서부터 들어오게해주시더라구요.\n매장이 꽤나 넓은편이라 저희도 다행히 5시다되서(한사이클 돌기전) 입장할 수 있었어요. \n빠른회전을 위해서인

In [ ]:
df['ko_text'][0]    ## 문장부호들 잘 걸러짐

'엄청 기대하고 갔는데이렇게 줄까지 서서 먹을만한 곳은 아니라는 생각이 들었습니다음엄청 달고 짜고그래도 떡은 진짜 맛있더라고요  볶음밥도 괜찮았습니다 좀 칼칼하고 깔끔한 닭볶음탕 찾으시는 분들은 다른 곳 추천드려요    지난번엔 줄이 길어 실패했는데 이번 방문엔 성공했네요양도 많고 맛있어요양념이 너무 맛있어서 밥 비벼 먹으면 최고추운날 얼큰한 국물 생각날때 좋아요기본 양이 많아 라면사리랑 볶음밥을 못 먹은게 너무 아쉽네요다 먹고 너무 맛있어서 집에 포장도 해왔어요 벌써 두번째 방문또간집 보고 방문했어요소스가 정말 미쳤어요 너무 맛있어요ㅠ다른 테이블에서도 다들 감탄소리뿐였어요닭에 소스가 베이는 동안 먹는 떡도 최고예오밀떡을 좋아하는데 간이 쏙쏙 베어들어 아주 환상이랍니다  감자도 정말 맛나는 감자예요 소스에 으깨먹으면 진짜 환장네이션닭살들도 정말 야들하고 냄새 안나요사장님 직원들 너무나도 친절하시구 좋습니다볶음밥은 꼭 드시고 가셔야 해요여기 참기름을 좋은 걸 쓰시는지 고소한 맛과 소스와 밥이 아주 환상적 볶음밥이랑 깍두기를 꼭 드셔야 해요 이 볶음밥을 위해 만들어진 깍두기인지 정말 찰떡궁합이랍니다 콩나물무침은 말할 것도 없거요 정말 강남 최고의 맛집 추천입니다단 테이블링은 꼭 하고 가셔야 해요 수요일 저녁 시 분에 테이블링 원격줄서기 걸고 시에 입장해서 먹었어요 텁텁함 도 없이 깔끔하고 맛있는 떡도리탕이었어요 약간 떡볶이 느낌이 더강한 감자전은 감자튀김전같아서 조금 실망했네요ㅠㅠ 밑반찬 콩나물이 증맬 맛있어요 또간집 나오기전에도 왔었는데 맛도리인걸 알았지만 또간집에까지 나와서 이렇게 많아질줄몰랐어요주말 시반정도방문했는데 앞에 팀 대기있었고 시까지 브레이크타임이었지만 시 분좀넘어서부터 들어오게해주시더라구요매장이 꽤나 넓은편이라 저희도 다행히 시다되서한사이클 돌기전 입장할 수 있었어요 빠른회전을 위해서인지 음식이 앉기도 전에 먼저 셋팅이되어서 앉자마자 바로 끓여서 먹을 수 있어서 더 좋았어요떡은 진짜 말캉거리고 닭은 잡내안나고 떡볶이맛인데 칼칼하면서 달달하고 그냥 

In [ ]:
## 띄어쓰기 교정

spacing = Spacing()
df['ko_text'] = df['ko_text'].apply(lambda x:spacing(x))

In [ ]:
df['ko_text'][0]

'엄청 기 대하고 갔는데 이렇게 줄까지 서서 먹을 만한 곳은 아니라는 생각이 들었습니다음 엄청 달고 짜고 그래도 떡은 진짜 맛있더라고요 볶음밥도 괜찮았습니다 좀 칼칼하고 깔끔한 닭볶음탕 찾으시는 분들은 다른 곳 추천 드려 요 지난번엔 줄이 길어 실패했는데 이번 방문엔 성공했네요 양도 많고 맛있어 요양념이 너무 맛있어서 밥 비벼 먹으면 최고 추운 날 얼큰한 국물 생각날 때 좋아요 기본 양이 많아 라면 사리랑 볶음밥을 못 먹은 게 너무 아쉽네요 다 먹고 너무 맛있어서 집에 포장도 해왔어요 벌써 두 번째 방문 또 간집 보고 방문했어요 소스가 정말 미쳤어요 너무 맛있어요ㅠ 다른 테이블에서도 다들 감탄소리뿐였어요닭에 소스가 베이는 동안 먹는 떡도 최고 예오밀 떡을 좋아하는데 간이 쏙쏙 베어들어 아주 환상이랍니다 감자도 정말 맛 나는 감자예 요 소 스에 으깨 먹으면 진짜 환장 네이션 닭살들도 정말 야들하고 냄새 안나요 사장님 직원들 너무나도 친절하시 구 좋습니다 볶음밥은 꼭 드시고 가셔야 해요 여기 참기름을 좋은 걸 쓰시는지 고소한 맛과 소스와 밥이 아주 환상적 볶음밥이랑 깍두기를 꼭 드셔야 해요 이 볶음밥을 위해 만들어진 깍두기인지 정말 찰떡궁합이랍니다 콩나물 무침은 말할 것도 없거요 정말 강남 최고의 맛집  추천입니다단 테이블링은 꼭 하고 가셔야 해요 수요일 저녁 시 분에 테이블링 원격 줄서기 걸 고 시에 입장해서 먹었어요 텁텁함 도 없이 깔끔하고 맛있는 떡도 리탕이었어요 약간 떡볶이 느낌이 더 강한 감자전은 감자튀김 전 같아서 조금 실망했네요 ㅠㅠ 밑반찬 콩나물이 증맬 맛있어요 또 간집 나오기 전에도 왔었는데 맛도리인걸 알았지만 또 간집에까지 나와서 이렇게 많아질 줄 몰랐어요 주말 시 반 정도 방문했는데 앞에 팀 대기 있었고 시까지 브레이크 타임이었지만 시 분 좀 넘어서부터 들어오게 해주시더라구 요 매장이 꽤나 넓은 편이라 저희도 다행히 시 다 되서 한 사이클 돌기 전 입장할 수 있었어요 빠른 회전을 위해서인지 음식이 앉기도 전에 먼저 셋팅이 되어서 앉자마자 바로 

In [ ]:
## spacing 오래 걸려서 csv 저장

# df.to_csv("/content/drive/MyDrive/AIVLE_BigProject/data/total_spacing.csv", encoding="utf-8-sig", index=False)

## ☆ 데이터 다시 로드

In [37]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/AIVLE_BigProject/data/total_spacing.csv")

In [38]:
df.shape

(12724, 55)

## 2-3. 텍스트 데이터 전처리
* 불용어 처리

In [23]:
temp = df['ko_text'].iloc[:2000] ## 용량 부족인가뵈

In [24]:
temp[0]

'엄청 기 대하고 갔는데 이렇게 줄까지 서서 먹을 만한 곳은 아니라는 생각이 들었습니다음 엄청 달고 짜고 그래도 떡은 진짜 맛있더라고요 볶음밥도 괜찮았습니다 좀 칼칼하고 깔끔한 닭볶음탕 찾으시는 분들은 다른 곳 추천 드려 요 지난번엔 줄이 길어 실패했는데 이번 방문엔 성공했네요 양도 많고 맛있어 요양념이 너무 맛있어서 밥 비벼 먹으면 최고 추운 날 얼큰한 국물 생각날 때 좋아요 기본 양이 많아 라면 사리랑 볶음밥을 못 먹은 게 너무 아쉽네요 다 먹고 너무 맛있어서 집에 포장도 해왔어요 벌써 두 번째 방문 또 간집 보고 방문했어요 소스가 정말 미쳤어요 너무 맛있어요ㅠ 다른 테이블에서도 다들 감탄소리뿐였어요닭에 소스가 베이는 동안 먹는 떡도 최고 예오밀 떡을 좋아하는데 간이 쏙쏙 베어들어 아주 환상이랍니다 감자도 정말 맛 나는 감자예 요 소 스에 으깨 먹으면 진짜 환장 네이션 닭살들도 정말 야들하고 냄새 안나요 사장님 직원들 너무나도 친절하시 구 좋습니다 볶음밥은 꼭 드시고 가셔야 해요 여기 참기름을 좋은 걸 쓰시는지 고소한 맛과 소스와 밥이 아주 환상적 볶음밥이랑 깍두기를 꼭 드셔야 해요 이 볶음밥을 위해 만들어진 깍두기인지 정말 찰떡궁합이랍니다 콩나물 무침은 말할 것도 없거요 정말 강남 최고의 맛집  추천입니다단 테이블링은 꼭 하고 가셔야 해요 수요일 저녁 시 분에 테이블링 원격 줄서기 걸 고 시에 입장해서 먹었어요 텁텁함 도 없이 깔끔하고 맛있는 떡도 리탕이었어요 약간 떡볶이 느낌이 더 강한 감자전은 감자튀김 전 같아서 조금 실망했네요 ㅠㅠ 밑반찬 콩나물이 증맬 맛있어요 또 간집 나오기 전에도 왔었는데 맛도리인걸 알았지만 또 간집에까지 나와서 이렇게 많아질 줄 몰랐어요 주말 시 반 정도 방문했는데 앞에 팀 대기 있었고 시까지 브레이크 타임이었지만 시 분 좀 넘어서부터 들어오게 해주시더라구 요 매장이 꽤나 넓은 편이라 저희도 다행히 시 다 되서 한 사이클 돌기 전 입장할 수 있었어요 빠른 회전을 위해서인지 음식이 앉기도 전에 먼저 셋팅이 되어서 앉자마자 바로 

In [25]:
okt = Okt()
words = " ".join(str(temp.tolist()))
words = okt.morphs(words,stem=True)

In [26]:
remove_one_word = [x for x in words if len(x)>1]
len(remove_one_word)

177244

In [ ]:
## temp[:500] 형태소 분석 결과

frequent = Counter(remove_one_word).most_common()

In [ ]:
frequent

[('늘다', 13573),
 ('가다', 12001),
 ('있다', 11950),
 ('서다', 10586),
 ('하다', 8791),
 ('지다', 7699),
 ('먹다', 7168),
 ('좋다', 5956),
 ('들다', 4725),
 ('오다', 3230),
 ('끄다', 2349),
 ('대다', 2279),
 ('같다', 2176),
 ('크다', 2090),
 ('많다', 1960),
 ('짜다', 1916),
 ('자다', 1671),
 ('되다', 1609),
 ('없다', 1545),
 ('않다', 1320),
 ('보다', 1266),
 ('밉다', 1221),
 ('볶다', 1143),
 ('갈다', 1009),
 ('싶다', 966),
 ('넘다', 856),
 ('튀다', 783),
 ('깔다', 660),
 ('이다', 648),
 ('겠다', 597),
 ('기다', 576),
 ('차다', 524),
 ('따다', 476),
 ('싸다', 408),
 ('받다', 397),
 ('꾸다', 377),
 ('찾다', 328),
 ('끼다', 324),
 ('맞다', 321),
 ('넓다', 305),
 ('켜다', 294),
 ('쓰다', 277),
 ('돼다', 276),
 ('푸다', 271),
 ('새다', 261),
 ('찌다', 239),
 ('내다', 226),
 ('치다', 199),
 ('써다', 192),
 ('추다', 192),
 ('쉬다', 191),
 ('넣다', 191),
 ('뜨다', 183),
 ('타다', 172),
 ('세다', 168),
 ('잇다', 167),
 ('놓다', 164),
 ('째다', 154),
 ('희다', 149),
 ('메다', 144),
 ('멀다', 139),
 ('덮다', 134),
 ('주다', 133),
 ('허다', 130),
 ('나다', 128),
 ('앉다', 127),
 ('굳다', 111),
 ('몰다', 109),
 ('쏘다', 103),
 ('높다'

In [ ]:
## Okt 모델로 형태소 분석. 빈도수로 많이 나오는 단어들 체크 후, 불용어 파일에 추가

In [39]:
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text) 
    return result


def load_stopwords():
    with open('/content/drive/MyDrive/AIVLE_BigProject/data/stopwords4.txt', 'r') as f:
        list_file = f.readlines()
    return list_file[0].split(",")


def make_wordlist(reviews,stopwords): #reviews = " ".join(data['Review'].tolist())
    print("리뷰들을 모아 분석하는 중입니다.....")
    #정규표현식 적용
    print("데이터 정제 중....")
    words = extract_word(reviews)
    #형태소 추출
    print("형태소 추출 중....")
    okt = Okt()
    words = okt.morphs(words,stem=True)
    #한글자 제거
    print("한글자 제거 중....")
    words = [x for x in words if len(x)>1]
    #불용어 제거
    print("불용어 제거 중....")
    words = [x for x in words if x not in stopwords]
    #최소횟수 미만 제거
    print("의미있는 단어리스트 생성 중....")
    time.sleep(1)
    minimum_count = 3
    final = []
    for i in tqdm(range(len(words))):
        tmp = words[i]
        if words.count(tmp) >= minimum_count:
            final.append(tmp)
    return set(final) #조건을 만족하는 단어 리스트


def preprocess(text, word_list):
    text = extract_word(text)
    okt = Okt()
    text = okt.morphs(text, stem = True)
    text = [x for x in text if x in word_list]
    return list(set(text))

In [40]:
stopwords = load_stopwords()

In [52]:
word_lists = []
for i in range(len(df)):
    word_list = make_wordlist(str(df['ko_text'][i]), stopwords)
    word_lists.append(word_list)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
### test

# word_lists = []
# for i in range(10):
#     word_list = make_wordlist(str(temp[i]), stopwords)
#     word_lists.append(word_list)

In [53]:
len(word_lists)

12724

In [55]:
for i in word_lists:
    print(i)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
{'좋다', '맛있다'}
{'고기', '수육', '국밥', '깔끔하다', '맛있다', '좋다', '국물', '친절하다', '돼지국밥'}
{'맛있다', '곰탕', '맛집', '연남동'}
{'한식', '코리아', '좋다', '맛있다'}
{'곰탕', '김치', '맛있다'}
{'비냉', '맛있다', '냉면', '좋다', '따뜻하다', '거홍', '육전', '교육'}
{'방어', '청어', '모듬', '맛있다', '좋다', '스끼'}
{'맛있다', '족발', '순대', '좋다', '친절하다', '포장'}
{'주류', '분위기', '어울리다', '특별하다', '추천', '한식', '맛있다', '수을관', '좋다', '전통주', '친절하다', '고급'}
{'곱창', '막창', '맛있다', '치즈', '홍대', '좋다', '눈꽃'}
{'곱창', '웨이팅', '좋다', '맛있다'}
{'서비스', '고기', '볶음밥', '등심', '맛있다', '손님', '먹음'}
{'소금', '계란', '맛있다', '치즈', '예약', '좋다', '양념'}
{'최고', '분위기', '곱창', '맛있다', '좋아하다', '맛집', '좋다', '친절하다'}
{'해장국', '좋다', '맛있다'}
{'메기', '맛있다', '막회', '채소', '신선하다'}
{'코스', '매운탕', '좋다', '맛있다'}
{'서비스', '분위기', '맛있다', '좋다', '친절하다'}
{'합정', '좋다', '맛있다', '분위기'}
{'버거', '맛있다', '햄버거', '좋다', '패티'}
{'분위기', '닭다리', '맛있다', '피클', '건지다', '좋다', '데이트'}
{'인테리어', '크림', '파스타', '맛있다', '웨이팅', '맛집', '좋다', '연남동', '친절하다', '바질', '트러플'}
{'커리', '카이', '와인', '뇨끼', '크림', '파스타', '추천', '맛있다', '고등어', '즐겁다', '부드럽다', '좋다', '친절하

In [ ]:
for i in word_lists:
    if i==set():
        print(i)

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

result = []
## tf-idf 기준 상위 n_top 개수 키워드 추출
for i in word_lists:
    if i==set():
        result.append('no result')
    else:
        tfidf_vectorizer = TfidfVectorizer()
        tf_idf_vect = tfidf_vectorizer.fit_transform(i).toarray()

        n_top = 5
        importance = np.argsort(np.asarray(tf_idf_vect.sum(axis=0)).ravel())[::-1]
        tfidf_feature_names = np.array(tfidf_vectorizer.get_feature_names())
        result.append(tfidf_feature_names[importance[:n_top]])
        # print(tfidf_feature_names[importance[:n_top]])
        # print('-'*200)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# result = []
# ## tf-idf 기준 상위 n_top 개수 키워드 추출
# for i in word_lists:

#     tfidf_vectorizer = TfidfVectorizer()
#     tf_idf_vect = tfidf_vectorizer.fit_transform(i).toarray()

#     n_top = 5
#     importance = np.argsort(np.asarray(tf_idf_vect.sum(axis=0)).ravel())[::-1]
#     tfidf_feature_names = np.array(tfidf_vectorizer.get_feature_names())
#     result.append(tfidf_feature_names[importance[:n_top]])
#     # print(tfidf_feature_names[importance[:n_top]])
#     # print('-'*200)

In [60]:
len(result)

12724

In [61]:
df['keywords'] = result

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12724 entries, 0 to 12723
Data columns (total 56 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   search_key            12724 non-null  object 
 1   name_x                12724 non-null  object 
 2   category              12724 non-null  object 
 3   rating                12724 non-null  float64
 4   address               12724 non-null  object 
 5   tel                   11152 non-null  object 
 6   year_10               12724 non-null  float64
 7   year_20               12724 non-null  float64
 8   year_30               12724 non-null  float64
 9   year_40               12724 non-null  float64
 10  year_50               12724 non-null  float64
 11  year_60               12724 non-null  float64
 12  male                  12724 non-null  int64  
 13  female                12724 non-null  int64  
 14  detail_url            12724 non-null  object 
 15  gu                 

In [68]:
df.to_csv("/content/drive/MyDrive/AIVLE_BigProject/review_keyword_total.csv", index=False, encoding="utf-8-sig")

In [69]:
df_test = pd.read_csv("/content/drive/MyDrive/AIVLE_BigProject/review_keyword_total.csv")

In [75]:
df_test.iloc[500:550, :]

,search_key,name_x,category,rating,address,tel,year_10,year_20,year_30,year_40,...,review_blog4,review_blog5,review_blog6,review_blog7,review_blog8,review_blog9,review_blog10,reviews,ko_text,keywords
500,중식,윤가 양꼬치 훠궈 역삼점,양꼬치,4.66,서울 강남구 언주로94길 9-4,02-565-8373,0.00,0.00,0.00,0.00,...,역삼역 맛집 윤가에서 무한 양꼬치 구이와 맥주 콜라보!|안뇽하세요오~~ 올해들어서 ...,너무 맛있는 강남역 양꼬치 맛집|이제 진짜 가을이 지나고 겨울이 오는지 날도 쌀쌀하...,[역삼동 양꼬치] 윤지양꼬치|얼마전에 건대 양꼬치거리에서 부모님이랑 양꼬치를 먹었는...,"선릉 역삼 / 양꼬치 무한리필 맛집, 윤가|회사동료들과 퇴근 후 찾게 된 양꼬치집 ...",[역삼동] 양꼬치를 마음껏 즐길 수 있는 양꼬치 무한리필 윤지양꼬치... |요즘 1...,"[선릉, 역삼 술집] 윤가 양꼬치 훠궈 역삼점 | 양꼬치 무한리필, 훠궈... |안...",흡족했던 선릉역 술집|주말에 친구들과 술 한잔 하러 선릉역 술집에 다녀왔어요. 잡내...,사장님께서 \n친절하시고 \n단체로 가니깐 서비스까지 많이 주셨습니다\n또 방문의사...,사장님께서 친절하시고 단체로 가니깐 서비스까지 많이 주셨습니다 또 방문 의사 있습니...,['친절하다' '차갑다' '중국' '좋다' '불리하다']
501,중식,신사양꼬치 신사본점,양꼬치,4.40,서울 강남구 강남대로152길 33,0507-1402-0216,0.00,0.00,0.00,0.00,...,10월 1주차 주간일기|하위 드디어 10월 일기 레쭈고임 냉굴이랑 같이 쿠팡 뛰자며...,9.5~9.11|1. 생일 한 줄 요약: 휴가 쓸걸........ 아 나 사실 생일...,•신사양꼬치• 신사맛집 본점 양꼬치 양갈비|신사양꼬치 신사본점 22.09.28 오늘...,"[가로수길 맛집] 신사동 가로수길 ""신사 양꼬치""에서 양꼬치 먹었어요~|안녕하세요!...",[신사·가로수길] 신사양꼬치 신사본점 :: 육즙 터지는 매콤한 양꼬치와... |안녕...,신사 양꼬치! 양꼬치는 칭다오와 함께~|여전히 오늘도 맛있게 먹기 위해 맛집 냠 사...,"[신사양꼬치] 신사역 - 양꼬치엔 칭따오|위치: 신사역, 세로수길 영업시간 : 17...",양꼬치집인데 양갈비는 좋아하지만 양꼬치는 별로 좋아하지않아서 가지요리와 오이무침 먹...,양꼬치집인데 양갈비는 좋아하지만 양꼬치는 별로 좋아하지 않아서 가지 요리와 오이무침...,['좋다' '오이' '맛있다' '꼬치' '갈비']
502,중식,마봉양꼬치 매봉점,양꼬치,4.52,서울 강남구 논현로38길 46-9 1층 마봉양고치,0507-1403-5509,0.00,0.00,0.00,0.00,...,마봉 뚱뚱한 양꼬치와 옥면 꿔바로우 매봉역맛집|뚱뚱한 양꼬치와 옥면 꿔바로우 매봉역...,"양재맛집, 매봉역맛집 통통한 양꼬치를 맛볼 수 있는 마봉양꼬치 매봉점|즐거운 화요일...",[양재/매봉] 숙성 양고기 양재맛집 마봉양꼬치 매봉점|양꼬치 육즙에 놀라고 온 숙성...,[양재맛집] 잡내없는 숙성 양고기로 양꼬치 입문매장 추천! 마봉양꼬치... |#양재...,매봉 맛집 마봉양꼬치 추천합니다|주말에 가족 외식으로 가끔 가는곳이예요. 아이들 데...,매봉역맛집 마봉양꼬치 매봉점 양재맛집|정말 오랜만에 양꼬치를 먹기 위해 방문한 매봉...,마봉양꼬치 매봉점에서 저녁 먹기~!추천!|주말에 유산소 운동을 하겠다며 열심히 걸었...,직원분들이 정말 친절하시고 열심히 일하셔서 갈 때마다 기분이 좋은 곳이에요. 양꼬...,직원 분들이 정말 친절하시고 열심히 일하셔서 갈 때마다 기분이 좋은 곳이에요 양꼬치...,['친절하다' '좋다' '연유' '신선하다' '맛있다']
503,중식,향방,중식당,4.45,서울 강남구 강남대로150길 10 2층,0507-1364-8785,0.00,0.00,0.00,0.00,...,향방의 해물누룽지탕.|지금은 신사동에 있는 중국요리집 향방. 중국요리 전문가인 이향...,"차돌 짬뽕 먹고 싶을 때 일부러 찾아가는 곳이란, 향방 -신사동 중국집|일부러 찾아...",향방|#향방 #중화요리 #신사역맛집 중국집 향방 오랜만에 방문하였습니다 유산슬 고추...,"신사역 중식당, 향방|괜히 밥 먹을곳을 찾아 헤맬때가 있다. 아무거나 먹으면 되긴 ...","신사동 향방 중국집 방문기|맛은 괜찮음, 오래된집, 2층에 위치, 주차는 어려울듯 ...",신사역 맛집 이향방 맛있는 후기~!|신사역에는 숨은 맛집이 정말 많지만 중국집으로는...,"신사역 중식 맛집, 향방 추천해요|안녕하세요? 수영입니다:) 신사역 중식 맛집, 향...",차돌짬뽕밥 핵. 강. 추 👍😋 안맵고 좋아용 갑자기 짬뽕국물에 밥 먹고싶었는데 \...,차돌짬뽕밥 핵 강 추 안 맵 고 좋아용 갑자기 짬뽕 국물에 밥 먹고 싶었는데 와서 ...,['차돌' '짬뽕' '좋다' '맛있다']
504,중식,텐파마라탕 마라샹궈,중식당,4.57,서울 강남구 압구정로79길 49 1층,02-511-2128,0.00,0.00,0.00,0.00,...,청담 텐파)텐파 마라탕 마라샹궈|니하오. 동생김이다하오. 1주에 최소 1회 마라탕섭...,청담마라탕 청담동마라샹궈 덴파 청담중국음식 중식당추천|#청담저녁메뉴 #내돈내먹 고민...,눈을 감으면 사랑이 보여|블로그 쓰는데 불려나가서 치킨머금 윗 여울 ㅠ_ㅠ 현진이의...,청담 텐파마라탕 마라샹궈 ★★★☆|갑자기 모이게 된 하프 서른하나 스물 여덟 모임 ...,청담역 맛집 텐파마라탕 코엑스 강소농대전 첫날 저녁|어제부터 코엑스에서 강소농대전을...,"[청담맛집] 청담동 마라탕을 찾으신다면 ""텐파 마라탕""추천해요!|안녕하세요! 두덩이...",221122 is what? What is 221122? 221122 is L.O....,마라탕이랑 꿔바로우 너무 맛있어요😃😃 마라샹궈~꿔바로우~맛집 맛집~ 맛있어요~ 근데...,마라탕이랑 꿔바로우 너무 맛있어요 마라샹궈 꿔 바로 우맛집 맛집 맛있어요 근데 마라...,['작년' '맛있다']
505,중식,춘리마라탕 강남CGV점,중식당,4.50,서울 강남구 강남대로 438 CGV 3층,02-555-6388,0.00,0.00,0.00,0.00,...,[강남마라탕추천] 강남춘리마라탕 | 강남마라탕맛집... |오랜만에 대학교 때 친했던...,신논현역 강남역 cgv 3층 영화 춘리마라탕 훠궈 맵기 3단계 점심... |해결하지...,주간 일기 :: 청계산 (리숨두부 • 청계다방) / 신도림 세계단|22. 11-2 ...,[강남/신논현] 존맛 그 잡채인 마라탕 춘리마라탕 내돈내산 후기|여러분 안녕하세요 ...,네모네모빔을 맞아라.|강남에 약속이 있어서 갔는데 생각보다 일찍도착하기도 하구 친구...,강남역 맛있는 마라탕 '춘리마라탕'|마라탕이 한창 유행할때는 안먹어보고 뒤늦게 마라...,다로미 첫 마라탕 강남CGV 춘리마라탕|마라탕이 한창 들어오기 시작했을 때 위생문제...,최애마라탕 거짓말 안보태고 진짜 50번은 넘게 온 곳.\n사장님도 얼굴알고 계실정도...,최애마라탕 거짓말 안보태고 진짜 번은 넘게 온 곳 사장님도 얼굴 알고 계실 정도 임...,['탕집' '좋다' '우도' '신선하다' '맛있다']
506,중식,마오 청담동점,중식당,4.07,서울 강남구 도산대로56길 17,02-514-8803,0.00,0.00,0.00,0.00,...,청담 합리적인 중국 가정식 | 마오|' 중국 가정식을 연상케하는 이국적인 인테리어에...,강남구 청담동 / [마오 청담동점] - 올드패션의 미학 Feat. 청담... |#마...,"[청담동] 마오 (MAO), 사천요리 전문점, 중국에서 먹는 듯한 중식당... |2...",마오 청담동점|안녕하세요 오늘소개드릴곳은 서울특별시 강남구 도산대로 56길 17 서...,강남구 청담동 북경식 오리구이 중국집 ㅡ#마오 MAO|어제 하는일 없이 집에 가만히...,마오 청담동점에서 가성비 좋은 북경오리(베이징덕)에 가볍게 회식|아니 진심 빨리 먹...,마오 청담점 북경오리 맛집 중국여행 느낌이 물씬